In [1]:
!pip install tensorflow


Processing /home/harish/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd/gast-0.2.2-cp36-none-any.whl
  Found existing installation: gast 0.3.2
    Uninstalling gast-0.3.2:
      Successfully uninstalled gast-0.3.2


In [2]:
!pip install tqdm

In [3]:
!pip install tflearn

In [4]:
!pip install --upgrade pip

Requirement already up-to-date: pip in /home/harish/anaconda3/lib/python3.6/site-packages (19.3.1)


In [5]:
!pip install opencv-python

In [6]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # a nice pretty percentage bar for tasks. Thanks to viewer Daniel BA1/4hler for this suggestion

TRAIN_DIR = 'train1'
TEST_DIR = 'test1'
IMG_SIZE = 50
LR = 1e-3
MODEL_NAME = 'sadvshappy-{}-{}.model'.format(LR, '2conv-basic')

In [7]:
!pip install wrapt --upgrade --ignore-installed

Processing /home/harish/.cache/pip/wheels/d7/de/2e/efa132238792efb6459a96e85916ef8597fcb3d2ae51590dfd/wrapt-1.11.2-cp36-cp36m-linux_x86_64.whl


In [8]:
!pip install tensorflow

In [9]:
def label_img(img):
    word_label = img.split('.')[-3]
    # conversion to one-hot array [cat,dog]
    #                            [much cat, no dog]
    if word_label == 'happy': return [1,0]
    #                             [no cat, very doggo]
    elif word_label == 'sad': return [0,1]

In [55]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data2.py', training_data)
    return training_data


In [56]:
def process_test_data():
    testing_data = []
  
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data2.py', testing_data)
    return testing_data

In [57]:
train_data = create_train_data()

100%|██████████| 80/80 [00:00<00:00, 234.76it/s]


In [48]:
import tensorflow as tf

In [14]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)


convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

train1 = train_data[:-35]
test1 = train_data[-35:]

X = np.array([i[0] for i in train1]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train1]

test_x = np.array([i[0] for i in test1]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test1]

model.fit({'input': X}, {'targets': Y}, n_epoch=70, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)










Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

---------------------------------
Run id: sadvshappy-0.001-2conv-basic.model
Log directory: log/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 45
Validation samples: 35
--
Training Step: 1  | time: 2.910s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0

Training Step: 23  | total loss: 0.56103 | time: 1.090s
| Adam | epoch: 023 | loss: 0.56103 - acc: 0.7943 | val_loss: 0.68902 - val_acc: 0.5429 -- iter: 45/45
--
Training Step: 24  | total loss: 0.54477 | time: 1.091s
| Adam | epoch: 024 | loss: 0.54477 - acc: 0.7771 | val_loss: 0.69559 - val_acc: 0.5143 -- iter: 45/45
--
Training Step: 25  | total loss: 0.51960 | time: 1.081s
| Adam | epoch: 025 | loss: 0.51960 - acc: 0.8319 | val_loss: 0.71308 - val_acc: 0.7143 -- iter: 45/45
--
Training Step: 26  | total loss: 0.50248 | time: 1.083s
| Adam | epoch: 026 | loss: 0.50248 - acc: 0.8705 | val_loss: 0.83237 - val_acc: 0.7143 -- iter: 45/45
--
Training Step: 27  | total loss: 0.46579 | time: 1.082s
| Adam | epoch: 027 | loss: 0.46579 - acc: 0.8809 | val_loss: 0.77559 - val_acc: 0.7714 -- iter: 45/45
--
Training Step: 28  | total loss: 0.45246 | time: 1.087s
| Adam | epoch: 028 | loss: 0.45246 - acc: 0.8663 | val_loss: 0.77627 - val_acc: 0.6000 -- iter: 45/45
--
Training Step: 29  | total l

Training Step: 69  | total loss: 0.77417 | time: 1.088s
| Adam | epoch: 069 | loss: 0.77417 - acc: 0.8883 | val_loss: 1.40067 - val_acc: 0.5714 -- iter: 45/45
--
Training Step: 70  | total loss: 0.68823 | time: 1.138s
| Adam | epoch: 070 | loss: 0.68823 - acc: 0.9012 | val_loss: 1.78966 - val_acc: 0.5714 -- iter: 45/45
--


In [15]:

import matplotlib.pyplot as plt

# if you need to create the data:
test_data = process_test_data()   #returnnig testing_data
# if you already have some saved:
#test_data = np.load('train_data.npy')

fig=plt.figure()

for num,data in enumerate(test_data[:10]):
    # cat: [1,0]
    # dog: [0,1]
    
    img_num = data[1]
    img_data = data[0]
    
    y = fig.add_subplot(3,4,num+1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
    #model_out = model.predict([data])[0]
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 1: str_label='sad'
    else: str_label='happy'
        
    y.imshow(orig,cmap='gray')
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()

100%|██████████| 30/30 [00:00<00:00, 137.87it/s]


<Figure size 640x480 with 10 Axes>

In [32]:
print(data[0][0])

[195]


In [16]:
with open('submission_file.csv','w') as f:
    f.write('id,label\n')
            
with open('submission_file3.csv','a') as f:
    for data in tqdm(test_data):
        img_num = data[1]
        img_data = data[0]
        orig = img_data
        data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
        model_out = model.predict([data])[0]
        f.write('{},{}\n'.format(img_num,model_out[1]))

100%|██████████| 30/30 [00:00<00:00, 203.77it/s]


In [19]:
import pickle

In [23]:
pickle.dump(model, open('model.pkl','wb'))

# Loading model to compare the results
model1 = pickle.load(open('model.pkl','rb'))

TypeError: can't pickle _thread.RLock objects